# Generative AI Search with Local LLM

free colab works. Mistral openhermes works and i love teky's balls of steel giving diamond level dataset which makes these openhermes models. Is what i tested with. Works.

 very easy to edit the code and make this for arxiv or whatever else. good stuff.

In [ ]:
# Install llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

%cd /content
!apt-get update -qq && apt-get install -y -qq aria2

# Download a local large language model, I'm using OpernHermes-2.5-Mistral-7B-16K-GGUF which has a longer context size and has pretty good quality at its size
# If you want to use other local models that can easily run on consumer hardware, check out this repo: https://github.com/Troyanovsky/Local-LLM-Comparison-Colab-UI/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF/resolve/main/openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf?download=true -d /content/model/ -o openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf

In [ ]:
import requests
import subprocess
import json
import time

# Function for calling search API
def get_search_results(search_term, max_retries=2, retry_delay=2):
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": search_term})
    headers = {
        'X-API-KEY': '<your_api_key>', # get an API Key stuck here
        'Content-Type': 'application/json'
    }

    retries = 0
    while retries < max_retries:
        try:
            response = requests.request("POST", url, headers=headers, data=payload)
            response.raise_for_status()  # Raise an exception for non-2xx status codes
            data = response.json()
            organic_results = data.get("organic", [])

            search_results = []
            search_results_str = ""
            index = 0
            for result in organic_results:
                title = result.get("title", "")
                link = result.get("link", "")
                snippet = result.get("snippet", "")
                search_results.append({"title": title, "link": link, "snippet": snippet})
                formatted_result = f"index: {index}\ntitle: {title}\nlink: {link}\nsnippet: {snippet}\n\n"
                search_results_str += formatted_result
                index += 1
            return search_results, search_results_str
        except requests.exceptions.RequestException as e:
            retries += 1
            print(f"Error: {e}. Retrying in {retry_delay} seconds... (Attempt {retries}/{max_retries})")
            time.sleep(retry_delay)

    raise Exception("Maximum retries exceeded. Failed to retrieve search results.")


def fetch_url_content(url):
    # this is the good thinguie that makes the html llm friendly
    prefixed_url = f"https://r.jina.ai/{url}"


    try:
        curl_cmd = [
            "curl",
            "-H",
            "Accept: text/event-stream",
            prefixed_url,
        ]
        curl_process = subprocess.Popen(curl_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = curl_process.communicate()

        if curl_process.returncode == 0:
            content = stdout.decode("utf-8")

            content_lines = [line for line in content.split("\n") if line.startswith("data: ")]
            if content_lines:
                content_data = "\n".join(line[6:] for line in content_lines)
                try:
                    content_value = json.loads(content_data)["content"]
                    return content_value
                except (ValueError, KeyError):
                    pass

            return ""
        else:
            error_message = stderr.decode("utf-8")
            raise Exception(f"cURL request failed: {error_message}")

    except Exception as e:
        raise Exception(f"An error occurred: {e}")

In [ ]:
# mans go local. Ran on my brazilian notebook. Probably works for u

from llama_cpp import Llama

def load_llama():
    llm = Llama(
            model_path="/content/model/openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf", # If you're using another model, change the name
            chat_format="chatml", # Use the chat_format that matches the model
            n_gpu_layers=-1, # Use -1 for all layers on GPU
            n_ctx=12288 # Set context size
    )
    return llm

def call_llama(input: str, llm) -> str:
    llm = llm
    output = llm.create_chat_completion(
        messages=[
            {
                "role": "system",
                "content": "You're a helpful assistant.",
            }, # Feel free to modify the prompt to suit your own formatting needs
            {"role": "user", "content": input},
        ],
        temperature=0.7,
    )
    output_text = output['choices'][0]['message']['content']
    return output_text

In [ ]:
def pick_url(query, search_results_str, search_results, llm):
    llm = llm
    prompt = f"Given the following question, which of the following URLs is most likely to contain the answer for it? Reply ONLY the index number. Question: ```{query}``` List: ```{search_results_str}```"
    index = call_llama(prompt, llm)

    max_retries = 2
    retries = 0
    while retries < max_retries:
        try:
            index = int(index.strip())
            break
        except ValueError:
            retries += 1
            index = call_llama(prompt, llm)

    if retries == max_retries:
        raise Exception("Failed to convert index to a valid integer after multiple retries.")

    try:
        return index
    except IndexError:
        raise Exception(f"Invalid index {index} for the search results list.")

In [ ]:
def search_with_ai(user_input):
    llm = None

    llm = load_llama()

    search_term_prompt = f"Based on the following question, plesae come up with a search term to use in the search engine. Reply the search term only. Quesiton: ```{user_input}```"
    search_term = call_llama(search_term_prompt, llm)
    print(f"Searching: {search_term}")

    # Seach with search API
    search_results, search_results_str = get_search_results(search_term)

    # Pick the most relevant URL
    try:
        top_url_index = pick_url(user_input, search_results_str, search_results, llm)
    except Exception as e:
        print(f"Error picking URL: {e}")
        return

    # Fetch the content from the top URL
    try:
        top_url = search_results[top_url_index]["link"]
        top_snippet = search_results[top_url_index]["snippet"]
        print(f"Crawling: {top_url}")
        content = fetch_url_content(top_url)
    except Exception as e:
        print(f"Error fetching URL content: {e}")
        del llm
        return

    # Truncate the content if it's longer than 36864 characters. I'm using a very lazy estimate here. You can count actual tokens instead.
    if len(content) > 36864:
        content = content[:36864]

    # Call LLM with the content and get the answer. This is the shit original prompt. If u actually use it i can make it way better. Or stun. He would make it way better x10
    answer_prompt = f"Answer the question from the given content. Question: ```{user_input}```\n\nContent:```From URL: {top_url} Snippet: {top_snippet}\n{content}```"
    try:
        answer = call_llama(answer_prompt, llm)
        return answer
    except Exception as e:
        print(f"Error calling LLM: {e}")
        return

In [ ]:
question = input("What is your question? \n")
answer = search_with_ai(question)
print(answer)